# Audioerkennung für Hörgeschädigte

## Data Collection
Es wurden insgesamt XYZ Minuten Audioaufnahmen für die zu klassifizierenden Kategorien "Laufendes Wasser", "Klopfen", WEITERE KATEGORIEN gesammelt. Für die Kategorie "Laufendes Wasser" wurden x Minuten Audiomaterial aufgenommen, für ...
Dabei wurden X verschiedene Geräte benutzt.

## Data Preparation

In [71]:
# Notwendige Bibliotheken importieren
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
import random
import streamlit as st
import matplotlib.pyplot as plt
import numpy as np

# Funktion zum Einlesen der CSV-Dateien
df_Water = pd.read_csv('Water.csv')
df_Knocking = pd.read_csv('Knocking.csv')
df_Water.drop(['time'], axis=1, inplace=True)
df_Knocking.drop(['time'], axis=1, inplace=True)

# Label hinzufügen
df_Water['Label'] = 'Water'
df_Knocking['Label'] = 'Knocking'


# TODO: Checke ob hierdurch Informationsverlust
# Auf ganze Zahl runden
df_Water['dBFS'] = df_Water['dBFS'].round(decimals = 0)
df_Knocking['dBFS'] =  df_Knocking['dBFS'].round(decimals = 0)

# Daten in einzelne Samples aufteilen
# TODO Kontrolliere: stimmt das??
# print(round(len(df_Water)/300))
split_Water = np.array_split(df_Water, 1000)
split_Knocking = np.array_split(df_Knocking, 1000)


# print(split_Water)

# Zusammenführen der beiden Dataframes (noch Water hinten und Knock vorne)

combined_df = []
for df in split_Knocking: 
    combined_df.append(df) 
    # Hinzufügen der DataFrames aus 'array2' zu 'combined_list' 
for df in split_Water: 
    combined_df.append(df)


# Blockweise mischen
# TODO das funktioniert noch nicht.
random.shuffle(combined_df)
print(combined_df)




[     seconds_elapsed  dBFS     Label
588           68.779     0  Knocking
589           68.899   -72  Knocking
590           69.017   -99  Knocking,       seconds_elapsed  dBFS  Label
4758          489.540 -27.0  Water
4759          489.641 -26.0  Water
4760          489.741 -27.0  Water
4761          489.841 -26.0  Water
4762          489.958 -27.0  Water
4763          490.058 -28.0  Water,      seconds_elapsed  dBFS  Label
768           78.700 -34.0  Water
769           78.801 -34.0  Water
770           78.901 -34.0  Water
771           79.001 -35.0  Water
772           79.101 -34.0  Water
773           79.201 -34.0  Water,       seconds_elapsed  dBFS     Label
1095          127.967  -109  Knocking
1096          128.082  -113  Knocking
1097          128.196     0  Knocking,       seconds_elapsed  dBFS  Label
5400          555.590 -29.0  Water
5401          555.691 -31.0  Water
5402          555.790 -34.0  Water
5403          555.890 -35.0  Water
5404          555.991 -35.0  Water
54

In [39]:

# Funktion zum Erstellen von Samples aus CSV-Dateien. Ein Sample besteht aus bestimmte Länge (sample_size) aufeinanderfolgender Dezibel-Werte. Es wird sichergestellt, dass keine Samples den gleichen Ausschnitt abbilden.
def process_csv_file(csv_file, sample_size, num_samples):
    samples = []
    used_samples = set()

    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Überspringe die Header-Zeile

        lines = list(reader)
        lines_without_first_column = [line[1:] for line in lines]

        total_lines = len(lines_without_first_column)
        max_start_index = total_lines - sample_size

        if max_start_index <= 0:
            print("Nicht genügend Zeilen in der CSV-Datei.")
            return

        while len(samples) < num_samples:
            start_index = random.randint(0, max_start_index)
            sample = lines_without_first_column[start_index: start_index + sample_size]
            sample_str = str(sample)

            if sample_str not in used_samples:
                label = None
                if 'Knocking' in csv_file:
                    label = 'Knock'
                elif 'Water' in csv_file:
                    label = 'Water'
                samples.append((sample, label))
                used_samples.add(sample_str)

    print(f"Anzahl der einzigartigen Samples: {len(samples)}")
    return samples

# Festlegen der Parameter sample_size num_samples für die Funktion process_csv_file
sample_size = 50
num_samples = 300

# TODO: Erstellen einer großen CSV je Kategorie 

# read Knocking file
knocking_df = pd.read_csv('Knocking.csv')
knocking_df.drop([[0,1]], axis=1, inplace=True)
print(knocking_df.head())

# Erstellen der Samples aus den CSV-Dateien
knock_data = process_csv_file('Knocking.csv', sample_size, num_samples)
water_data = process_csv_file('Water.csv', sample_size, num_samples)

KeyError: '[(0, 1)] not found in axis'

## Modeling

## Evaluation